In [ ]:
### Grading script code 
### You don't need to read this, proceed to the next cell
import sys
import functools
ipython = get_ipython()

def set_traceback(val):
    method_name = "showtraceback"
    setattr(
        ipython,
        method_name,
        functools.partial(
            getattr(ipython, method_name),
            exception_only=(not val)
        )
    )

class AnswerError(Exception):
  def __init__(self, message):
    pass

def exec_test(f, question):
    try:
        f()
        print(question + " Pass")
    except:
        set_traceback(False) # do not remove
        raise AnswerError(question + " Fail")

# Week 9 Problem Set

## Homeworks

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

**HW0.** Copy and paste some of the functions from you cohort sessions and previous exercises that you will need in this homework. See below template and the list here:
- `normalize_z()`
- `get_features_targets()`
- `calc_linear()`
- `prepare_feature()`
- `prepare_target()`
- `predict()`
- `split_data()`
- `r2_score()`
- `mean_squared_error()`

Then do the following:
- Read the CSV file `housing_processed.csv` and extract the following columns:
    - x data: RM, DIS, and INDUS columns 
    - y data: MEDV column
- Normalize the features using z normalization. 
- Plot the data using scatter plot. Use the following columns:


In [ ]:
def normalize_z(df=pd.DataFrame):
    df_mean = df.mean(axis=0)
    df_std = df.std(axis=0)
    return (df - df_mean)/df_std

def get_features_targets(df, feature_names, target_names):
    return pd.DataFrame(df.loc[:, feature_names]), pd.DataFrame(df.loc[:, target_names])

def prepare_feature(df_feature):
    # feature is the X, so u need a column of 1s in the 1st column
    df_feature.insert(0, 'ones', 1)
    return df_feature.to_numpy()
    

def prepare_target(df_target):
    return df_target.to_numpy()

# get predicted y values through y = X * b_hat formula
'''
this function should standardize the feature using z normalization, change it to a Numpy array, and add a column of constant 1s. 
You should use `prepare_feature()` for this purpose. 
'''
def predict(df_feature, beta):
    return calc_linear(prepare_feature(normalize_z(df_feature)), beta)

def calc_linear(X, beta):
    return X @ beta

def split_data(df_feature, df_target, random_state=None, test_size=0.5):
    if random_state:
        np.random.seed(random_state)
    indexes = df_feature.index
    k = int(len(indexes) * test_size)
    test_index = set(np.random.choice(indexes, size=k, replace=False))
    train_index = set(indexes) - test_index
    
    df_feature_train = df_feature.loc[train_index, :]
    df_feature_test = df_feature.loc[test_index, :]
    df_target_train = df_target.loc[train_index, :]
    df_target_test = df_target.loc[test_index, :]
    return df_feature_train, df_feature_test, df_target_train, df_target_test
  
# do it according to formula: 1 - SSres/SStot
# SSres = summation(y_i - y_i_hat); actual y - predicted y
# SStot = summation(y_i - y_mean); actual y - mean y
def r2_score(y, ypred):
    y_mean = np.mean(y)
    y_ypred = y - ypred
    y_ymean = y - y_mean
    SS_res = y_ypred.T @ y_ypred
    SS_tot = y_ymean.T @ y_ymean
    
    return 1 - SS_res/SS_tot


# MSE = 1/n * summation(actual y - predicted y)
def mean_squared_error(target, pred):
    y_diff = target - pred
    return 1/target.shape[0] * y_diff.T @ y_diff

In [ ]:
# Read the CSV file
df = pd.read_csv('housing_processed.csv')

# Extract the features and the targets
df_features, df_target = get_features_targets(df, ['RM', 'DIS', 'INDUS'], 'MEDV')

# Normalize using z normalization
df_features = normalize_z(df_features)

###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
display(df_features.describe())
display(df_target.describe())
assert np.isclose(df_features['RM'].mean(), 0)
assert np.isclose(df_features['DIS'].mean(), 0)
assert np.isclose(df_features['INDUS'].mean(), 0)

assert np.isclose(df_features['RM'].std(), 1)
assert np.isclose(df_features['DIS'].std(), 1)
assert np.isclose(df_features['INDUS'].std(), 1)

assert np.isclose(df_target['MEDV'].mean(), 22.532806)
assert np.isclose(df_target['MEDV'].std(), 9.197104)

assert np.isclose(df_features['RM'].median(), -0.1083583)
assert np.isclose(df_features['DIS'].median(), -0.2790473)
assert np.isclose(df_features['INDUS'].median(), -0.2108898)

In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
sns.set()
plt.scatter(df_features["RM"], df_target)

In [ ]:
plt.scatter(df_features["DIS"], df_target)

In [ ]:
plt.scatter(df_features["INDUS"], df_target)

**HW1.** *Multiple variables cost function:* Write a function `compute_cost_multi()` to compute the cost function of a linear regression model. The function should take in two 2-D numpy arrays. The first one is the matrix of the linear equation and the second one is the actual target value.

Recall that:

$$J(\hat{\beta}_0, \hat{\beta}_1) = \frac{1}{2m}\Sigma^m_{i=1}\left(\hat{y}(x^i)-y^i\right)^2$$

where

$$\hat{y}(x) =  \hat{\beta}_0 + \hat{\beta}_1 x_1 + \hat{\beta}_2 x_2 + \ldots + \hat{\beta}_n x_n$$

The function should receive three Numpy arrays:
- `X`: is the feature 2D Numpy array
- `y`: is the target 2D Numpy array
- `beta`: is the parameter 2D Numpy array

The function should return the cost which is a float.

You can use the following function in your code:
- `np.matmul(array1, array2)`

Note that if you wrote your Cohort session's `compute_cost()` using proper Matrix operations to do the square and the summation, the code will be exactly the same here.

In [ ]:
def compute_cost(X, y, beta):
    y_hat = calc_linear(X, beta)
    y_hat_minus_y = y_hat - y
    m = X.shape[0]
    J = 1/(2*m) * y_hat_minus_y.T @ y_hat_minus_y
    return J[0,0]


In [ ]:
X = prepare_feature(df_features)
target = prepare_target(df_target)

assert isinstance(X, np.ndarray)
assert isinstance(target, np.ndarray)
assert X.shape == (506, 4)
assert target.shape == (506, 1)

In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
beta = np.zeros((4,1))
J = compute_cost(X, target, beta)
print(J)
assert np.isclose(J, 296.0734)

beta = np.ones((4,1))
J = compute_cost(X, target, beta)
print(J)
assert np.isclose(J, 270.4083)

beta = np.array([-1, 2, 1, 2]).reshape((4,1))
J = compute_cost(X, target, beta)
print(J)
assert np.isclose(J, 314.8510)


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW2.** *Gradient Descent:* Write a function called `gradient_descent_multi()` that takes in four parameters:
- `X`: is a 2-D numpy array for the features
- `y`: is a vector array for the target
- `alpha`: is the learning rate
- `num_iters`: is the number of iteration to perform

The function should return two arrays:
- `beta`: is coefficient at the end of the iteration
- `J_storage`: is the array that stores the cost value at each iteration

You can use some of the following functions:
- `np.matmul(array1, array2)`: which is to do matrix multiplication on two Numpy arrays.
- `compute_cost()`: which the function you created in the previous problem set to compute the cost.

Note that if you use proper matrix operations in your cohort sessions for the gradient descent function, the code will be the same here.

In [ ]:
def gradient_descent(X, y, beta, alpha, num_iters):
    J = []
    for iter in range(num_iters):
        y_hat = calc_linear(X, beta)
        beta = beta - alpha * 1/X.shape[0] * X.T @ (y_hat - y)
        J.append(compute_cost(X, y, beta))
    return beta, J


In [ ]:
iterations = 1500
alpha = 0.01
beta = np.zeros((4,1))

beta, J_storage = gradient_descent(X, target, beta, alpha, iterations)
print(beta)
assert np.isclose(beta[0], 22.5328)
assert np.isclose(beta[1], 5.4239)
assert np.isclose(beta[2], -0.90367)
assert np.isclose(beta[3], -2.95818)

In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
plt.plot(J_storage)

**HW3.** Do the following tasks:
- Get the features and the targets.
    - features: RM, DIS, INDUS
    - target: MEDV
- Use the previous functions called `predict()` to calculated the predicted values given the features and the model. 
- Create a target numpy array from the data frame.

In [ ]:
# This is from the previous result
beta = np.array([[22.53279993],
       [ 5.42386374],
       [-0.90367449],
       [-2.95818095]])

# Extract the feature and the target
df_features, df_target = get_features_targets(df, ['RM', 'DIS', 'INDUS'], 'MEDV')
# Call predict() 
pred = predict(df_features, beta)

# Change target to numpy array
target = prepare_target(df_target)

###
### YOUR CODE HERE
###



In [ ]:
assert isinstance(pred, np.ndarray)
assert np.isclose(pred.mean(), 22.5328)
assert np.isclose(pred.std(), 6.7577)

In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
plt.scatter(df_features["RM"],target)
plt.scatter(df_features["RM"],pred)

In [ ]:
plt.scatter(df_features["DIS"],target)
plt.scatter(df_features["DIS"],pred)

In [ ]:
plt.scatter(df_features["INDUS"],target)
plt.scatter(df_features["INDUS"],pred)

**HW4.** *Splitting data:*  Do the following tasks:
- Extract the following:
    - features: RM, DIS, and INDUS
    - target: MEDV 
- Use Week 9's function `split_data()` to split the data into train and test using `random_state=100` and `test_size=0.3`.
- Normalize and prepare the features and the target.
- Use the training data set and call `gradient_descent()` to obtain the `theta`.
- Use the test data set to get the predicted values.

You need to replace the `None` in the code below with other a function call or any other Python expressions. 

In [ ]:
# Extract the features and the target
df_features, df_target = get_features_targets(df, ['RM', 'DIS', 'INDUS'], 'MEDV')

# Split the data set into training and test
df_features_train, df_features_test, df_target_train, df_target_test = split_data(df_features, df_target, random_state=100, test_size=0.3)

# Normalize the features using z normalization
df_features_train_z = normalize_z(df_features_train)

# Change the features and the target to numpy array using the prepare functions
X = prepare_feature(df_features_train_z)
target = prepare_target(df_target_train)

iterations = 1500
alpha = 0.01
beta = np.zeros((4,1))

# Call the gradient_descent function
beta, J_storage = gradient_descent(X, target, beta, alpha, iterations)

# call the predict() method
pred = predict(df_features_test, beta)
###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
assert isinstance(pred, np.ndarray)
assert pred.shape == (151, 1)
assert np.isclose(pred.mean(), 22.66816)
assert np.isclose(pred.std(), 6.67324)

In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
plt.scatter(df_features_test["RM"], df_target_test)
plt.scatter(df_features_test["RM"], pred)

In [ ]:
plt.scatter(df_features_test["DIS"], df_target_test)
plt.scatter(df_features_test["DIS"], pred)

In [ ]:
plt.scatter(df_features_test["INDUS"], df_target_test)
plt.scatter(df_features_test["INDUS"], pred)

**HW5.** Calculate the coefficient of determination, $r^2$.

In [ ]:
# change target test set to a numpy array
target = prepare_target(df_target_test)

# Calculate r2 score by calling a function
r2 = r2_score(target, pred)

###
### YOUR CODE HERE
###

print(r2)


In [ ]:
assert np.isclose(r2, 0.47713)

In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW6.** Calculate the mean squared error.

In [ ]:
# Calculate the mse
mse = mean_squared_error(target, pred)

###
### YOUR CODE HERE
###
print(mse)


In [ ]:
assert np.isclose(mse, 51.363)

In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW7.** *Polynomial Transformation:* Redo the steps for breast cancer data but this time we will use quadratic model. Use the following columns:
- x data: radius_mean
- y data: area_mean

We will create a quadratic hypothesis for this x and y data. To do that write a function `transform_features(df, colname, colname_transformed)` that takes in a dataframe for the features, the original column name, and the transformed column name. The function should add another column with the name `colname_transformed` with the value of column in `colname` transformed to its quadratic value. 

In [ ]:
# Read from breast_cancer_data.csv file
df = pd.read_csv('breast_cancer_data.csv')

# Extract feature and target
df_feature, df_target = get_features_targets(df, 'radius_mean', 'area_mean')

###
### YOUR CODE HERE
###

plt.scatter(df_feature, df_target)

In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
# write your function to create a quadratic feature of 

'''
We will create a quadratic hypothesis for this x and y data.
To do that write a function `transform_features(df, colname, colname_transformed)` 
that takes in a dataframe for the features, the original column name, and the transformed column name. 
The function should add another column with the name `colname_transformed` with the value of column in `colname` 
transformed to its quadratic value. 
'''

def transform_features(df_feature, colname, colname_transformed):
    # add a column and multiply df_Feature[colname] squared
    df_feature[colname_transformed] = df_feature[colname]**2
    return df_feature


In [ ]:
df_features = transform_features(df_feature, "radius_mean", "radius_mean^2")

assert np.allclose(df_features.loc[:,"radius_mean^2"], df_features.loc[:,"radius_mean"] ** 2)

In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
# split data using random_state = 100 and 30% test size
df_features_train, df_features_test, df_target_train, df_target_test = split_data(df_features, df_target, random_state=100, test_size=0.3)

# normalize features
df_features_train_z = normalize_z(df_features_train)

# change to numpy array and append column for feature
X = prepare_feature(df_features_train_z)
target = prepare_target(df_target_train)

iterations = 1500
alpha = 0.01
beta = np.zeros((3,1))

# call gradient_descent() function
beta, J_storage = gradient_descent(X, target, beta, alpha, iterations)

###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:

assert np.isclose(beta[0], 646.0787)
assert np.isclose(beta[1], 146.4801)
assert np.isclose(beta[2], 201.9803)

In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
plt.plot(J_storage)

In [ ]:
# change target to numpy array
beta = np.array([[646.0787641 ], [146.4800792 ], [201.98031254]])

target = prepare_target(df_target_test)
pred = predict(df_features_test, beta)

# get predicted values
###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
plt.scatter(df_features_test["radius_mean"], target)
plt.scatter(df_features_test["radius_mean"], pred)

In [ ]:
target = prepare_target(df_target_test)
r2 = r2_score(target, pred)
print(r2)
assert np.isclose(r2, 0.985095)

In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
target = prepare_target(df_target_test)
mse = mean_squared_error(target, pred)
print(mse)
assert np.isclose(mse, 1919.164)

In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW8.** *Optional:* Redo the above tasks using Sci-kit learn libraries. You will need to use the following:
- [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)
- [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)
- [r2_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html)
- [mean_squared_error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html)
- [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

**Redo HW 4 using Scikit Learn**

In [ ]:
# Read the housing_processed.csv file
df = pd.read_csv('housing_processed.csv')

# extract the features from ["RM", "DIS", "INDUS"] and target from []"MEDV"]
df_features, df_target = get_features_targets(df, ['RM', 'DIS', 'INDUS'], 'MEDV')
# normalize
df_features = normalize_z(df_features)

###
### YOUR CODE HERE
###


In [ ]:
# Split the data into training and test data set using scikit-learn function
df_features_train, df_features_test, df_target_train, df_target_test = train_test_split(df_features, df_target, test_size=0.3, train_size=0.7, random_state=100)
# Instantiate LinearRegression() object
model = LinearRegression()

# Call the fit() method
model.fit(df_features_train, df_target_train)

###
### YOUR CODE HERE
###
print(model.coef_, model.intercept_)
assert np.isclose(model.coef_, [ 5.01417104, -1.00878266, -3.27301726]).all()
assert np.isclose(model.intercept_, 22.45962454)

In [ ]:
# Call the predict() method
pred = model.predict(df_features_test)

###
### YOUR CODE HERE
###


In [ ]:
plt.scatter(df_features_test["RM"], df_target_test)
plt.scatter(df_features_test["RM"], pred)

In [ ]:
plt.scatter(df_features_test["DIS"], df_target_test)
plt.scatter(df_features_test["DIS"], pred)

In [ ]:
plt.scatter(df_features_test["INDUS"], df_target_test)
plt.scatter(df_features_test["INDUS"], pred)

In [ ]:
r2 = r2_score(df_target_test, pred)
print(r2)
assert np.isclose(r2, 0.48250)

In [ ]:
mse = mean_squared_error(df_target_test, pred)
print(mse)
assert np.isclose(mse, 52.41451)

**Redo HW7 Using Scikit Learn**

In [ ]:
# Read the file breast_cancer_data.csv
df = pd.read_csv('breast_cancer_data.csv')
# extract feature from "radius_mean" and target from "area_mean"
df_feature, df_target = get_features_targets(df, 'radius_mean', 'area_mean')

###
### YOUR CODE HERE
###


In [ ]:
# # instantiate a PolynomialFeatures object with degree = 2
# poly = PolynomialFeatures(degree=2)

# # call its fit_transform() method
# df_features = poly.fit_transform(X=df_feature)

# # call train_test_split() to split the data
# df_features_train, df_features_test, df_target_train, df_target_test = train_test_split(df_features, df_target, train_size=0.7, test_size=0.3, random_state=100)

# # instantiate LinearRegression() object
# model = LinearRegression()

# # call its fit() method
# model.fit(df_features_train, df_target_train)


### BEGIN SOLUTON
poly = PolynomialFeatures(2)
df_features = poly.fit_transform(df_feature)
df_features_train, df_features_test, df_target_train, df_target_test = train_test_split(df_features, df_target, random_state=100, test_size=0.3)
model = LinearRegression()
model.fit(df_features_train, df_target_train)
### END SOLUTION
print(model.coef_, model.intercept_)
assert np.isclose(model.coef_, [0., 3.69735512, 2.9925278 ]).all()
assert np.isclose(model.intercept_, -32.3684598)

In [ ]:
# Call the predict() method
pred = model.predict(df_features_test)

###
### YOUR CODE HERE
###

print(type(pred), pred.mean(), pred.std())
assert isinstance(pred, np.ndarray)
assert np.isclose(pred.mean(), 672.508465)
assert np.isclose(pred.std(), 351.50271)

print(pred)

In [ ]:
plt.scatter(df_features_test[:,1], df_target_test)
plt.scatter(df_features_test[:,1], pred)
print(len(pred))

In [ ]:
r2 = r2_score(df_target_test, pred)
print(r2)
assert np.isclose(r2, 0.99729)

In [ ]:
mse = mean_squared_error(df_target_test, pred)
print(mse)
assert np.isclose(mse, 346.79479)